In [2]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    
    full_pred_matrix = np.dot(P, Q.T)
    
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse


In [10]:
def matrix_factorization(R, K, steps = 200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    
    np.random.seed(1)
    P = np.random.normal(scale = 1./K, size = (num_users, K))
    Q = np.random.normal(scale = 1./K, size = (num_items, K))
    
    break_count = 0
    
    non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]
    
    for step in range(steps):
        for i, j, r in non_zeros:
            eij = r - np.dot(P[i, :], Q[j, :].T)
            
            P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
            Q[j, :] = Q[j, :] + learning_rate * (eij * P[i, :] - r_lambda * Q[j, :])
        
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0:
            print('### iteration step : ', step, 'rmse : ', rmse)
    return P, Q

In [11]:
import pandas as pd
import numpy as np

movies = pd.read_csv('D:/머신러닝 완벽가이드 데이터/ml-latest-small/movies.csv')
ratings = pd.read_csv('D:/머신러닝 완벽가이드 데이터/ml-latest-small/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index = 'userId', columns = 'movieId')

# title 컬럼을 얻기 이해 movies 와 조인 수행
rating_movies = pd.merge(ratings, movies, on = 'movieId')

# columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = rating_movies.pivot_table('rating', index = 'userId', columns = 'title')

In [12]:
P, Q = matrix_factorization(ratings_matrix.values, K = 50, steps = 200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step :  0 rmse :  2.9023619751336867
### iteration step :  10 rmse :  0.7335768591017927
### iteration step :  20 rmse :  0.5115539026853442
### iteration step :  30 rmse :  0.37261628282537446
### iteration step :  40 rmse :  0.2960818299181014
### iteration step :  50 rmse :  0.2520353192341642
### iteration step :  60 rmse :  0.22487503275269854
### iteration step :  70 rmse :  0.20685455302331537
### iteration step :  80 rmse :  0.19413418783028685
### iteration step :  90 rmse :  0.18470082002720403
### iteration step :  100 rmse :  0.17742927527209104
### iteration step :  110 rmse :  0.1716522696470749
### iteration step :  120 rmse :  0.1669518194687172
### iteration step :  130 rmse :  0.16305292191997542
### iteration step :  140 rmse :  0.15976691929679643
### iteration step :  150 rmse :  0.1569598699945732
### iteration step :  160 rmse :  0.15453398186715428
### iteration step :  170 rmse :  0.15241618551077643
### iteration step :  180 rmse :  0.15055080739

In [18]:
ratings_pred_matrix = pd.DataFrame(data = pred_matrix, index = ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),127 Hours (2010),13 Assassins (Jûsan-nin no shikaku) (2010),13 Ghosts (1960),13 Going on 30 (2004),13 Hours (2016),13 Sins (2014),13 Tzameti (2005),13th (2016),"13th Warrior, The (1999)",1408 (2007),1492: Conquest of Paradise (1992),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1776 (1972),18 Again! (1988),187 (One Eight Seven) (1997),1900 (Novecento) (1976),1941 (1979),1969 (1988),...,Young People Fucking (a.k.a. YPF) (2007),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),"Young Victoria, The (2009)","Young and the Damned, The (Olvidados, Los) (1950)",Youngblood (1986),Your Friends and Neighbors (1998),Your Highness (2011),Your Name. (2016),Your Sister's Sister (2011),"Yours, Mine and Ours (1968)","Yours, Mine and Ours (2005)",Youth (2015),Youth Without Youth (2007),Youth in Revolt (2009),Youth of the Beast (Yaju no seishun) (1963),Z (1969),Zabriskie Point (1970),Zack and Miri Make a Porno (2008),Zapped! (1982),Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,1.623927,3.910138,4.775403,3.837260,3.875488,1.550801,2.929129,2.680321,3.225626,3.251925,2.778350,3.331543,2.391855,3.199047,4.148949,1.852731,3.269642,3.448719,4.458060,3.719499,3.231820,3.521511,3.866924,3.961768,4.957933,4.075665,3.509040,3.923190,3.210152,4.374122,4.410886,4.492122,3.363129,3.661694,4.668763,2.706866,3.499069,4.120021,3.906938,3.615929,2.287447,4.071106,3.455642,4.063553,3.400589,1.347731,2.951010,3.095293,3.069549,1.815583,...,2.470922,2.745355,3.031998,3.887258,3.638946,2.149185,3.003406,2.915339,3.822582,3.673548,3.098158,3.508259,3.907094,2.107600,4.087125,3.086592,4.323157,3.213992,4.686308,1.611300,3.546313,3.207635,2.082641,3.302390,1.821505,3.814172,4.227119,3.699006,3.009256,4.605246,4.712096,4.284418,3.095067,3.214574,0.990303,1.805794,4.588016,2.295002

In [19]:
def get_unseen_movies(ratings_matrix, userId):
    user_rating = ratings_matrix.loc[userId, :]
    
    already_seen = user_rating[user_rating > 0].index.tolist()
    
    movies_list = ratings_matrix.columns.tolist()
    
    unseen_list = [movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

In [20]:
def recomm_movie_by_userId(pred_df, userId, unseen_list, top_n = 10):
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending = False)[:top_n]
    return recomm_movies

In [21]:
unseen_list = get_unseen_movies(ratings_matrix, 9)

recomm_movies = recomm_movie_by_userId(ratings_pred_matrix, 9, unseen_list, top_n=10)

recomm_movies = pd.DataFrame(data = recomm_movies.values, index = recomm_movies.index, columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690
